In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import os

# --- Configuration ---
MODEL_NAME = "zhihan1996/DNABERT-2-117M"
DATA_PATH = "posneg_sequences.csv"   # ← 여기 수정됨
OUTPUT_PATH = "./results/embeddings.csv"

def load_model():
    print(f"Loading model: {MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True
    )
    model = AutoModel.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        attn_implementation="eager"   # 🔥 핵심
    )
    model.eval()
    return tokenizer, model

def get_embedding(sequence, tokenizer, model):
    inputs = tokenizer(
        sequence,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512
    )
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy().flatten()
    return embedding

def main():
    tokenizer, model = load_model()

    df = pd.read_csv(DATA_PATH)
    print("Generating embeddings...")

    embeddings = []
    for seq in df["sequence"]:
        embeddings.append(get_embedding(seq, tokenizer, model))

    df["embedding"] = embeddings

    os.makedirs("results", exist_ok=True)
    df.to_csv(OUTPUT_PATH, index=False)

    print(f"✅ Saved embeddings to {OUTPUT_PATH}")

if __name__ == "__main__":
    main()


Loading model: zhihan1996/DNABERT-2-117M


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

...

bert_padding.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- bert_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


flash_attn_triton.py: 0.00B [00:00, ?B/s]

Encountered exception while importing triton: No module named 'triton'


ImportError: This modeling file requires the following packages that were not found in your environment: triton. Run `pip install triton`

In [ ]:
pip install triton